In [ ]:
from pulp import *
import pandas as pd
import plotly.express as px

## Load in data

In [ ]:
def load_res_profile(file:str)->pd.Series:

    res_norm = pd.read_csv(f'data/renewable_profiles/{file}',skiprows=3,index_col=0)
    res_norm.index = pd.to_datetime(res_norm.index)
    res_norm = res_norm['electricity']
    return res_norm

wind_profile = 'wind_amsterdam.csv'
solar_profile = 'solar_amsterdam.csv'

wind = load_res_profile(wind_profile).iloc[:2]
solar = load_res_profile(solar_profile).iloc[:2]

## Define load and renewables size (MW)

In [ ]:
load = 80
solar_cap = 100
wind_cap = 200

res = (wind * wind_cap) + (solar * solar_cap)
res.name = 'Renewable Production'
res

In [ ]:
df = pd.DataFrame(res)
df['Total Load'] = load
df['LoL initial'] = df['Total Load'] - df['Renewable Production']
df.describe()

In [ ]:
lol_mean =df['LoL initial'].mean()
if lol_mean > 0:
    raise Exception(f'Dataset has less res supply than load, solve is impossible. loss of load average: {lol_mean}')


In [ ]:
px.line(df['LoL initial'],title='Loss of load (negative means excess supply)')

In [ ]:
plot_cols = ['Total Load','Renewable Production','LoL initial']
px.line(df.groupby(df.index.month)[plot_cols].sum(),title='Monthly loss of load')

## Load constants

In [ ]:
#res = res_series.values.tolist()

In [ ]:
time_periods = range(df.shape[0])
time_periods

In [ ]:
#Power must at least be able to match worst loss of load event in pre-storage setup
power_min = df['LoL initial'].max()
power_min

## Define linear problem

In [ ]:
model = LpProblem(name="battery", sense=LpMinimize)


### Define scalar variables

In [ ]:
capacity = LpVariable(name='capacity',lowBound=0)
power = LpVariable(name='power',lowBound=power_min) #power_min

### Define time-dependent variables

In [ ]:
level_i = LpVariable.dicts(name='storagelevel',indices=time_periods,lowBound=0)
#level_i

In [ ]:
dischrg_i = LpVariable.dicts(name='discharge',indices=time_periods,lowBound=0)
chrg_i = LpVariable.dicts(name='charge',indices=time_periods,lowBound=0)


## Assign scalar constants

In [ ]:
efficiency = 0.70
min_duration = 4


# Capex is the only consideration for cost
capex={
    'power':800,
    'capacity':100,
}


cost = capex

cost


## Define objective function

In [ ]:
obj_func_capex_total = capacity * cost['capacity'] + power * cost['power']

model += obj_func_capex_total

## Define constraints 

In [ ]:
constraint = capacity >= min_duration * power
model += constraint, "storage duration must be greater than 10"

In [ ]:
time_periods

In [ ]:
for t in time_periods:
    constraint = load <= res[t] + dischrg_i[t] - chrg_i[t]
    model += constraint, f"load is supplied (t={t})"


In [ ]:
for t in time_periods:
    constraint = chrg_i[t] <= power 
    model += constraint, f"storage charge is limited by power rating (t={t})"

In [ ]:
for t in time_periods:
    constraint = dischrg_i[t] <= power
    model += constraint, f"storage discharge is limited by power rating (t={t})"

In [ ]:
for t in time_periods:
    constraint = level_i[t] <= capacity
    model += constraint, f"storage level cannot be higher then capacity (t={t})"

In [ ]:
#initial_constraint = level_i[0] == 0#capacity
t_max = max(time_periods)
start_of_year_constraint = level_i[0] == level_i[t_max] - dischrg_i[t_max] + efficiency * chrg_i[t_max]
model += start_of_year_constraint, f"storage level at start is determined by the end"


In [ ]:
for t in time_periods[:-1]:
    constraint = level_i[t+1] == level_i[t] - dischrg_i[t] + efficiency * chrg_i[t]
    model += constraint, f"storage level is determined by charge and discharge, and efficiency (t={t})"
    del constraint

In [ ]:
for t in time_periods[:-1]:
    constraint = level_i[t] >= dischrg_i[t]
    model += constraint, f"cannot dispatch more than available energy (t={t})"
    del constraint

## Solve optimisation

In [ ]:
model.variables()

In [ ]:
status = model.solve()

In [ ]:
status

### Verify solve

In [ ]:
print(f"status: {model.status}, {LpStatus[model.status]}")
if model.status != 1:
    raise Exception('Model has not solved')


## Results

In [ ]:
for var in model.variables():
    if var.name in ['capacity','power']:
        print(f"{var.name}: {var.value()}")

In [ ]:
variables = {}
for v in model.variables():
    variables[v.name] = v.value()

#variables

In [ ]:
var_df = pd.Series(variables).reset_index()
var_df.rename(columns={0:'value'},inplace=True)

var_df['parameter'] = var_df['index'].str.split('_').str[0]
var_df

In [ ]:
time_results = var_df.loc[var_df['parameter'].isin(['storagelevel','charge','discharge'])].copy()

time_results['timestep'] = time_results['index'].str.split('_').str[-1]
time_results['timestep'] = pd.to_numeric(time_results['timestep'])

time_results['time'] = df.index[0]+pd.to_timedelta(time_results['timestep'] + start_date_hours, unit='H')
time_results=time_results.sort_values(by=['parameter','time'])

In [ ]:
plot_df = time_results.pivot(index='time',columns='parameter',values='value')
plot_df['charge_cumulative'] = plot_df['charge'].cumsum()
plot_df['discharge_cumulative'] = plot_df['discharge'].cumsum()


plot_df = pd.concat([plot_df,df['Renewable Production']],axis=1)
plot_df['Load'] = load
plot_df['Curtailment_final'] = plot_df['Renewable Production'] + plot_df['discharge'] - plot_df['charge'] - load 
plot_df.describe()

In [ ]:
px.line(plot_df,title=f'Storage dispatch. Power {round(variables["power"],0)} capacity {round(variables["capacity"],0)}')

In [ ]:
plot_df['net charge'] = -plot_df['charge']

generation_params = [
    'net charge',
    'discharge',
    'Renewable Production'
]
px.area(plot_df[generation_params],title='Generation minus storage charging')

In [ ]:
px.area(plot_df[generation_params].resample('1m').mean(),title='Monthly generation and storage averages')